# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [1]:
DATA_FOLDER = 'Data/microbiome/' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [2]:
# Write your answer here

## Task 2. RNA Sequences

### Problem

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.



### Resolution




By definition a microbiome is a set of microorganism in a particular environment. This term is widely used with human microbiome, witch has the goal to determine and classify all the microorganism who lives in the human body in order to determine his role in the human health and disease. In order to know the names of them, a high-throughput RNA sequencing procedure is done. This method is done several time with different environment setting, refer to the metadata file.  



As said above, there are 9 files that contained the trasciption of the RNA sequence and 1 metadata file. A good definition of the representation of the data, aswell abstract and practical, is very important because it gives a clear structure to the raw data. It is also easier to refer to it and to study any relations with other data representation. By inspecting the raw midi files we can see that it is structured as follow (taxon, count). An index must be choosen, lets see if the taxon are unique.


In [45]:
# Impor the libraries
import pandas as pd
import numpy as np


mb = pd.read_excel('Data/microbiome/MID1.xls', 'Sheet 1', index_col=0, header=None)


# Verify if The taxon index is uniq for a single file
print('uniqness of the index for a single file : ' + str(mb.index.is_unique))






#Create a numpy array of Dataframes. Note that we allow as there is only 10 files, we allow the for loop. 
# It is executed quickly.
for i in range(2,10): 
    filename = 'Data/microbiome/MID'+str(i)+'.xls'
    
    mbi = pd.read_excel(filename, 'Sheet 1', index_col=0, header=None)
    # Concatenate with the previous ones.

    mb=pd.concat([mb,mbi], axis=0) # concatenation by row.
    

# Name the index 
mb.columns = ['Count']
mb.index.name =  'Taxon'


# Test the uniquness of the taxon as index.
print('uniqness of the index for all the combined files : ' + str(mb.index.is_unique))


uniqness of the index for a single file : True
uniqness of the index for all the combined files : False


If we take Taxon as index, the index will not be unique, but we know that the index are unique for a single file. The metadata file has a unique index, namely the barcode. So, the idea is to combine the metadata and the data and use as unique index `barcode+taxon` .

In [171]:
metadata =  pd.read_excel('Data/microbiome/metadata.xls', 'Sheet1', index_col= 0 )

# Store the metadata index in order to create the hiearchical index.
metadataIndex = metadata.index


mb1 = pd.read_excel('Data/microbiome/MID1.xls', 'Sheet 1', index_col=0, header=None)
mb1.columns = ['Count']
mb1.index.name =  'Taxon'
mb1.index = indexing = mb1.index.str.split(' ', expand=True)

arrayMID = [mb1]

for i in range(2,10): 
    filename = 'Data/microbiome/MID'+str(i)+'.xls'
    
    mbi = pd.read_excel(filename, 'Sheet 1', index_col=0, header=None)
    
    mbi.columns = ['Count']
    mbi.index.name =  'Taxon'
    
    mbi.index = indexing = mbi.index.str.split(' ', expand=True)
    
    arrayMID.append(mbi)
    
    
mb =  pd.concat(arrayMID, keys=metadataIndex)

mb = mb.rename_axis(("BARCODE","Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "sd", "ds"))


mb = mb.join(metadata)
mb.fillna('Unknown')



Count  \
BARCODE Kingdom  Phylum                  Class                 Order                    Family                           Genus                sd  ds           
MID1    Archaea  "Crenarchaeota"         Thermoprotei          Desulfurococcales        Desulfurococcaceae               Ignisphaera          NaN NaN      7   
                                                                                        Pyrodictiaceae                   Pyrolobus            NaN NaN      2   
                                                               Sulfolobales             Sulfolobaceae                    Stygiolobus          NaN NaN      3   
                                                               Thermoproteales          Thermofilaceae                   Thermofilum          NaN NaN      3   
                 "Euryarchaeota"         "Methanomicrobia"     Methanocellales          Methanocellaceae                 Methanocella         NaN NaN      7   
                                                               Methanosarcinales        Methanosarcinaceae               Methanimicrococcus   NaN NaN      1   
                                                                                        Methermicoccaceae                Methermicoccus       NaN NaN      1   
                                         Archaeoglobi          Archaeoglobales          Archaeoglobaceae                 Ferroglobus          NaN NaN      1   
                                                                                                                         Geoglobus            NaN NaN      1   
                                         Halobacteria          Halobacteriales          Halobacteriaceae                 Haloplanus           NaN NaN      1   
                                                                                                                         Haloquadratum        NaN NaN      4   
                                                                                                                         Halosimplex          NaN NaN      1   
                                                                                                                         Natronobacterium     NaN NaN      2   
                                                                                                                         Natronomonas         NaN NaN      4   
                                         Methanococci          Methanococcales          Methanocaldococcaceae            Methanotorris        NaN NaN      1   
                                         Methanopyri           Methanopyrales           Methanopyraceae                  Methanopyrus         NaN NaN     12   
                                         Thermoplasmata        Thermoplasmatales        Thermoplasmatales_incertae_sedis Thermogymnomonas     NaN NaN      2   
        Bacteria "Actinobacteria"        Actinobacteria        Actinomycetales          Acidothermaceae                  Acidothermus         NaN NaN      2   
                                                                                        Beutenbergiaceae                 Salana               NaN NaN      1   
                                                                                        Bogoriellaceae                   Bogoriella           NaN NaN      1   
                                                                                        Cellulomonadaceae                Tropheryma           NaN NaN      1   
                                                                                        Corynebacteriaceae               Turicella            NaN NaN      2   
                                                                                        Cryptosporangiaceae              Cryptosporangium     NaN NaN     53   
                                                                                        Dermabacteraceae                 Dermabacter          NaN NaN  

The index are unique but the biological classification is not visible, in order to split the taxon we need to parse it. 

In [170]:
mb1 = pd.read_excel('Data/microbiome/MID1.xls', 'Sheet 1', index_col=0, header=None)
mb1.columns = ['Count']
mb1.index.name =  'Taxon'
#mb1.index
#mb1.index = mb1.index.str.extract('(A-Z)\w+',expand = True)
mb1.index = indexing = mb1.index.str.split(' ', expand=True)
#indexing.shape
mb1

    
mb1.rename_axis(("Kingdom", "Phylum", "Class", "Order", "Family", "Genus"))
#print(mb1)

Count
 Archaea "Crenarchaeota"          Thermoprotei          Desulfurococcales        Desulfurococcaceae               Ignisphaera                7
                                                                                 Pyrodictiaceae                   Pyrolobus                  2
                                                        Sulfolobales             Sulfolobaceae                    Stygiolobus                3
                                                        Thermoproteales          Thermofilaceae                   Thermofilum                3
 Archaea "Euryarchaeota"          "Methanomicrobia"     Methanocellales          Methanocellaceae                 Methanocella               7
                                                        Methanosarcinales        Methanosarcinaceae               Methanimicrococcus         1
                                                                                 Methermicoccaceae                Methermicoccus             1
                                  Archaeoglobi          Archaeoglobales          Archaeoglobaceae                 Ferroglobus                1
                                                                                                                  Geoglobus                  1
                                  Halobacteria          Halobacteriales          Halobacteriaceae                 Haloplanus                 1
                                                                                                                  Haloquadratum              4
                                                                                                                  Halosimplex                1
                                                                                                                  Natronobacterium           2
                                                                                                                  Natronomonas               4
                                  Methanococci          Methanococcales          Methanocaldococcaceae            Methanotorris              1
                                  Methanopyri           Methanopyrales           Methanopyraceae                  Methanopyrus              12
                                  Thermoplasmata        Thermoplasmatales        Thermoplasmatales_incertae_sedis Thermogymnomonas           2
 Bacteria "Actinobacteria"        Actinobacteria        Actinomycetales          Acidothermaceae                  Acidothermus               2
                                                                                 Beutenbergiaceae                 Salana                     1
                                                                                 Bogoriellaceae                   Bogoriella                 1
                                                                                 Cellulomonadaceae                Tropheryma                 1
                                                                                 Corynebacteriaceae               Turicella                  2
                                                                                 Cryptosporangiaceae              Cryptosporangium          53
                                                                                 Dermabacteraceae                 Dermabacter                1
                                                                                                                  Devriesea                  2
                                                                                 Dermatophilaceae                 Kineosphaera               3
                                                                                 Frankineae_incertae_sedis        Fodinicola                 1
                                                                                 Geodermatophilaceae              Blastococcus       

ValueError: Length of names must match number of levels in MultiIndex.

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here